# User input

You are free to change these parameters accordingly:

In [ ]:
dataset = '/content/drive/My Drive/Colab Notebooks/Project3/in_dataset.csv' # path to the dataset
queryset = '/content/drive/My Drive/Colab Notebooks/Project3/in_queryset.csv' # path to the queryset
output_dataset_file = '/content/drive/My Drive/Colab Notebooks/Project3/comp_dataset.csv' # path to the output_dataset_file
output_queryset_file = '/content/drive/My Drive/Colab Notebooks/Project3/comp_queryset.csv' # path to the output_queryset_file

dir = '/content/drive/My Drive/Colab Notebooks/Project3/Saves/' # the directory to which the model will be save or from which it will be loaded
model_name = 'compressor.h5' # file name of saved model (file must be of type .h5)

# Includes

In [ ]:
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, BatchNormalization, LSTM, RepeatVector
from keras.models import Model
from keras.models import model_from_json
from keras import regularizers
import datetime
import time
import requests as req
import json
import pandas as pd
import pickle
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Dataset loading and editing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print('Python version ' + sys.version)
print("---------------------------------------")

df = pd.read_csv(dataset, delimiter='\t', header=None)
df2 = pd.read_csv(queryset, delimiter='\t', header=None)
print(df)

df.rename(columns = {0:'id'}, inplace=True) #rename the first column to "id"
df2.rename(columns = {0:'id'}, inplace=True) #rename the first column to "id"
print(df)
ids = df['id'].values
ids2 = df2['id'].values
print(ids[0])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Python version 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
---------------------------------------
     0        1        2        3     ...    3647    3648    3649    3650
0       a  11.1440  10.4710  10.8280  ...   68.25   68.11   67.47   66.81
1      aa  47.4580  45.8700  46.2490  ...   46.97   45.89   43.33   43.01
2    aaba  13.8800  12.9700  13.2900  ...   72.22   71.16   70.19   70.56
3    aapl   1.2037   1.1589   1.1385  ...  174.18  175.61  175.25  174.67
4     abc  13.9910  13.4340  13.1950  ...   74.91   75.01   75.27   75.00
..    ...      ...      ...      ...  ...     ...     ...     ...     ...
354  xray  17.2850  16.9010  17.0160  ...   64.38   64.52   65.56   65.58
355   xrx  36.3880  35.2030  35.7070  ...   28.59   29.37   29.27   29.17
356   yum   8.2660   8.3172   8.2781  ...   81.01   81.13   79.53   79.59
357   zbh  46.3950  45.

Set seed for reproductability:

In [ ]:
import os
import random
import tensorflow

def reproducibleResults(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tensorflow.random.set_seed(seed)
    np.random.seed(seed)

reproducibleResults(12345)

# Reducing dimensionality via our model

Shuffle the dataset and the split it to test and test sets:

In [ ]:
df.drop("id", axis=1, inplace=True)

test = df

df2.drop("id", axis=1, inplace=True)

test2 = df2

print(test.shape)

(359, 3650)


Transpose the sets for our convinience:

In [ ]:
test = test.transpose()

test2 = test2.transpose()
print(test.head())

      0       1      2       3    ...     355     356     357     358
1  11.144  47.458  13.88  1.2037  ...  36.388  8.2660  46.395  47.669
2  10.471  45.870  12.97  1.1589  ...  35.203  8.3172  45.815  46.668
3  10.828  46.249  13.29  1.1385  ...  35.707  8.2781  42.181  46.570
4  10.698  47.852  13.09  1.1437  ...  35.104  8.3591  41.582  46.974
5  10.910  49.524  13.45  1.1679  ...  36.282  8.2781  43.444  47.004

[5 rows x 359 columns]


Perform scaling:

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))

for i in range(test.shape[1]):
  curr_ts = test[i].to_numpy()
  curr_ts = curr_ts.reshape(-1,1)
  test[i] = scaler.fit_transform(curr_ts)

for i in range(test2.shape[1]):
  curr_ts = test2[i].to_numpy()
  curr_ts = curr_ts.reshape(-1,1)
  test2[i] = scaler.fit_transform(curr_ts)

print(test.shape)

(3650, 359)


Split to X and y:

In [ ]:
import numpy as np

def create_dataset(X, time_steps=1, lag=1):
    Xs, ys = [], []
    for j in range(len(X.columns)):
      for i in range(0, len(X) - time_steps, lag):
          v = X[j].iloc[i:(i + time_steps)].values
          Xs.append(v)
          ys.append(X[j].iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
WINDOW = 50
LAG = 10

# reshape to (samples, window, n_features)

X_test, y_test = create_dataset(test, WINDOW, LAG)

X_test = X_test[:, :, np.newaxis]

X_test2, y_test2 = create_dataset(test2, WINDOW, LAG)

X_test2 = X_test2[:, :, np.newaxis]

print(X_test.shape)
print(y_test.shape)

(129240, 50, 1)
(129240,)


Make predictions:

In [ ]:
full_path = dir + model_name
model = keras.models.load_model(full_path)

decoded_stocks = model.predict(X_test)
decoded_stocks2 = model.predict(X_test2)
print(decoded_stocks.shape)

NameError: ignored

# Building outputs

In [ ]:
subseries_per_series = int(decoded_stocks.shape[0] / test.shape[1])
print(subseries_per_series)

First we need to recover the time series:

In [ ]:
concatenated_ts = []
for i in range(test.shape[1]):
    index = i*subseries_per_series
    curr_ts = decoded_stocks[index]
    for j in range(1, subseries_per_series):
        curr_ts = np.concatenate([curr_ts, decoded_stocks[index+j]])
    # print(len(curr_ts))
    concatenated_ts.append(curr_ts)
print(len(concatenated_ts))

subseries_per_series2 = int(decoded_stocks2.shape[0] / test2.shape[1])
print(subseries_per_series2)

concatenated_ts2 = []
for i in range(test2.shape[1]):
    index = i*subseries_per_series2
    curr_ts = decoded_stocks2[index]
    for j in range(1, subseries_per_series2):
        curr_ts = np.concatenate([curr_ts, decoded_stocks2[index+j]])
    # print(len(curr_ts))
    concatenated_ts2.append(curr_ts)
print(len(concatenated_ts2))

print(np.array(concatenated_ts).squeeze().shape)

In [ ]:
ids_df = pd.DataFrame(ids)
concatenated_ts_df = pd.DataFrame(np.array(concatenated_ts).squeeze())
output_df = pd.concat([ids_df, concatenated_ts_df], axis=1)
output_df = output_df.transpose()
output_df.reset_index(drop=True, inplace=True)
output_df = output_df.transpose()
output_csv = output_df.to_csv(output_dataset_file, index=False, header = False, sep ='\t', line_terminator = '\n')

ids_df2 = pd.DataFrame(ids2)
concatenated_ts_df2 = pd.DataFrame(np.array(concatenated_ts2).squeeze())
output_df2 = pd.concat([ids_df2, concatenated_ts_df2], axis=1)
output_df2 = output_df2.transpose()
output_df2.reset_index(drop=True, inplace=True)
output_df2 = output_df2.transpose()
output_csv2 = output_df2.to_csv(output_queryset_file, index=False, header = False, sep ='\t', line_terminator = '\n')